## Парсинг сайта [CCRL](https://computerchess.org.uk/ccrl/4040/)

Условия отбора модели:
- наличие тэгов `free` (бесплатная модель) или `oss` (модель с открытым исходным кодом)
- метрика `score`= wins + draws/2 >= 50%

Дополнительно было сделана сортировка по `LOS`, т.е. преимуществу рассматриваемой модели над остальными, которая определяется как LOS = P(Score > score  of 2 programs with equal strength)

Подробнее: [Math Statistic](https://www.chessprogramming.org/Match_Statistics#Likelihood_of_superiority), [LOS table](https://www.chessprogramming.org/LOS_Table)

In [ ]:
import requests
from bs4 import BeautifulSoup

def parse_chess_engines1() -> list:
    # Отправляем запрос на сайт и получаем HTML-страницу
    url = "https://computerchess.org.uk/ccrl/4040/"
    response = requests.get(url)
    html_content = response.content

    # Парсим HTML-страницу с помощью BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Находим таблицу с данными о шахматных движках
    table = soup.find("table", class_="rating_table")

    # Создаем список для хранения информации о движках
    engines_info = []

    table_rows = table.find_all("tr")[2:-1]

    # Обрабатываем каждую строку в таблице, не учитывая заголовок
    for row1, row2 in zip(table_rows[::2], table_rows[1::2]):
        # Находим все ячейки в строке
        cells = row1.find_all("td")
        # Извлекаем данные из ячеек
        engine_name = cells[1].get_text().strip()
        score = float(cells[5].get_text().strip().replace("%", ""))
        los = float(row2.find("td").get_text().strip().replace("%", ""))

        # Проверяем, что движок является бесплатным или с открытым исходным кодом
        if score >= 50 and cells[1].select("span.free, span.oss"):
            engines_info.append({
                "name": engine_name,
                "score": score,
                'LOS': los
            })

    # Сортируем список движков по убыванию счета
    engines_info.sort(key=lambda x: (x['score'], x['LOS']), reverse=True)

    return engines_info

engines = parse_chess_engines1()

# Вызываем функцию парсинга и выводим результаты
for i, engine in enumerate(engines):
    print(f"{i+1}-й движок - {engine['name']}")
    print(f"Счет: {engine['score']}%")
    print(f"Преимущество: {engine['LOS']}%")
    print()
print(f'Количество: {len(engines)}')

1-й движок - JSBam 0.52
Счет: 81.9%
Преимущество: 66.6%

2-й движок - Spartan 1.0
Счет: 63.5%
Преимущество: 72.9%

3-й движок - Philemon C
Счет: 62.3%
Преимущество: 60.8%

4-й движок - Stockfish 20230613 64-bit 4CPU
Счет: 61.8%
Преимущество: 95.4%

5-й движок - VanillaChess 2.6g
Счет: 61.5%
Преимущество: 99.8%

6-й движок - Mr Bob 1.3.0 64-bit
Счет: 59.2%
Преимущество: 52.3%

7-й движок - Matant 5.04
Счет: 58.3%
Преимущество: 57.6%

8-й движок - Dumb 2.1 64-bit
Счет: 57.8%
Преимущество: 52.0%

9-й движок - Mint 2.3
Счет: 57.7%
Преимущество: 53.7%

10-й движок - Clarity 6.0.0 64-bit
Счет: 57.5%
Преимущество: 60.0%

11-й движок - akimbo 1.0.0 64-bit
Счет: 57.3%
Преимущество: 71.9%

12-й движок - Jazz 840 64-bit 4CPU
Счет: 57.2%
Преимущество: 63.3%

13-й движок - Freyr 1.068 64-bit
Счет: 56.9%
Преимущество: 61.8%

14-й движок - SdBC 0.5.15.1
Счет: 56.6%
Преимущество: 62.6%

15-й движок - Armageddon 2.308
Счет: 56.6%
Преимущество: 59.8%

16-й движок - Grizzly 1.40.1b
Счет: 56.0%
Преимущест

Анализ вручную (не увенчался успехом):
1. JSBam 0.52 : нет в России
2. Spartan 1.0 : https://github.com/christiandaley/Spartan - нет лицензии
3. Philemon C : не возможно открыть\
...

[Словарь движков, откуда производился поиск инфы](https://www.chessprogramming.org/Engines)

**STOCKFISH** занимает:
- 4 место по `score` (61.8%)
- 3 место по `LOS` (95.4%)

## Парсинг ["12 Strong Free and Open Source Chess Engines"](https://www.linuxlinks.com/best-free-open-source-chess-engines/)

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def parse_chess_engines2() -> list:
    url = "https://www.linuxlinks.com/best-free-open-source-chess-engines/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    table = soup.find("table", id="tablepress-705")
    rows = table.find_all("tr")[1:]

    engines_info = []

    for row in rows:
        cells = row.find_all("td")
        engine_name = cells[0].text.strip()
        engine_link = cells[0].find("a")["href"]

        engine_info = {
            "name": engine_name,
            "UCI protocol": False,
            "website": '',
            "support": '',
            "license": ''
        }

        if engine_name != 'Andscacs':
            engine_info.update(parse_engine_page(urljoin(url, engine_link)))
        else:
            engine_info["website"] = engine_link

        engines_info.append(engine_info)

    return engines_info

def parse_engine_page(url: str) -> dict:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    engine_data = {}

    # Проверяем, есть ли упоминание об UCI протоколе
    uci_mention = soup.find(string=lambda text: "UCI" in text)
    engine_data["UCI protocol"] = uci_mention is not None

    # Извлекаем другую ключевую информацию
    website = soup.find(string="Website:")
    if website:
        engine_data["website"] = website.find_next('a')['href']

    support = soup.find(string="Support:")
    if support:
        support_link = support.find_next('a')['href']
        engine_data["support"] = support_link if not support_link.startswith('/') else ''

    license = soup.find(string="License:")
    if license:
        license_info = license.find_next(string=True).strip()
        engine_data["license"] = license_info

    return engine_data

info = parse_chess_engines2()

for i, engine_info in enumerate(info):
    print(f"{i+1} ENGINE: {engine_info['name']}")
    print(f"UCI protocol: {engine_info['UCI protocol']}")
    print(f"Website: {engine_info['website']}")
    print(f"Support: {engine_info['support']}")
    print(f"License: {engine_info['license']}")
    print()

1 ENGINE: Stockfish
UCI protocol: True
Website: https://stockfishchess.org/
Support: https://stockfishchess.org/blog/
License: GNU General Public License v3.0

2 ENGINE: Lc0
UCI protocol: True
Website: https://lczero.org/
Support: https://groups.google.com/g/lczero?pli=1
License: GNU General Public License v3.0

3 ENGINE: RubiChess
UCI protocol: True
Website: https://github.com/Matthies/RubiChess
Support: 
License: GNU General Public License v3.0

4 ENGINE: Nemorino
UCI protocol: True
Website: https://bitbucket.org/christian_g_nther/nemorino/src/master/
Support: 
License: GNU General Public License v3.0

5 ENGINE: Igel
UCI protocol: True
Website: https://github.com/vshcherbyna/igel
Support: 
License: GNU General Public License v3.0

6 ENGINE: Xiphos
UCI protocol: True
Website: https://github.com/milostatarevic/xiphos
Support: 
License: GNU General Public License v3.0

7 ENGINE: Laser
UCI protocol: True
Website: https://github.com/jeffreyan11/laser-chess-engine
Support: 
License: GNU Ge